<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/GenderRecognition/GenderRecognisionTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

In [10]:
!unzip -qq gender-recognition-200k-images-celeba.zip

In [11]:
import os
from tensorflow.keras import models,layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
width=height=224
batch_size=32

In [12]:
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)
train_data=data_generator.flow_from_directory(
    "Dataset/Train",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

val_data=data_generator.flow_from_directory(
    "Dataset/Validation",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
)

test_data=data_generator.flow_from_directory(
    "Dataset/Test",
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
)

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.


In [13]:
model_r=tf.keras.applications.ResNet50V2(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

94683136/94668760 [==============================] - 2s 0us/step


In [14]:
for layer in model_r.layers[:-2]:
  layer.trainable=False

In [15]:
model=tf.keras.Sequential([
        model_r,
        layers.Dense(2,activation='softmax'),
])

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="GenderDetection")
config = wandb.config
config.learning_rate = 0.001

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.learning_rate ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [19]:
model.fit(train_data,
          steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=10,
          callbacks=[WandbCallback()],

)

Epoch 1/10
5000/5000 [==============================] - 1107s 215ms/step - loss: 0.4189 - accuracy: 0.9148 - val_loss: 0.3799 - val_accuracy: 0.9224


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
5000/5000 [==============================] - 1068s 214ms/step - loss: 0.3799 - accuracy: 0.9232 - val_loss: 0.3489 - val_accuracy: 0.9322
Epoch 3/10
5000/5000 [==============================] - 1065s 213ms/step - loss: 0.3703 - accuracy: 0.9264 - val_loss: 0.3900 - val_accuracy: 0.9259
Epoch 4/10
5000/5000 [==============================] - 1064s 213ms/step - loss: 0.3894 - accuracy: 0.9264 - val_loss: 0.3389 - val_accuracy: 0.9364
Epoch 5/10
5000/5000 [==============================] - 1065s 213ms/step - loss: 0.3810 - accuracy: 0.9273 - val_loss: 0.5158 - val_accuracy: 0.9106
Epoch 6/10
5000/5000 [==============================] - 1064s 213ms/step - loss: 0.3898 - accuracy: 0.9271 - val_loss: 0.3252 - val_accuracy: 0.9360
Epoch 7/10
5000/5000 [==============================] - 1075s 215ms/step - loss: 0.3862 - accuracy: 0.9270 - val_loss: 0.4589 - val_accuracy: 0.9162
Epoch 8/10
5000/5000 [==============================] - 1067s 213ms/step - loss: 0.3852 - accuracy: 0.9274

In [22]:
model.save('gender.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [20]:
model.evaluate(test_data)

626/626 [==============================] - 120s 192ms/step - loss: 0.3140 - accuracy: 0.9364


[0.3139840066432953, 0.9364031553268433]

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[11189   353]
 [  956  7503]]
Classification Report
              precision    recall  f1-score   support

      Female       0.92      0.97      0.94     11542
        Male       0.96      0.89      0.92      8459

    accuracy                           0.93     20001
   macro avg       0.94      0.93      0.93     20001
weighted avg       0.94      0.93      0.93     20001

